### Forwards optimization notebook

By forward optimization here I mean optimization that only tunes existing parameters.
AKA finding the optimal value for any given parameter the strategy has so that it yields better results.

This notebook aims to create an interface json which can later be easily read for the optimal parameters for a given timeframe (start to date and period before to date).

for start_date in range(first_date, last_date):
    for end_date in range(start_date, last_date):
        generate_list of stuff
        
for thing in generated list not yet parsed:
    run the computation cronjob
    save the parsed config for later
    
def compute_optimal_stuff():
    filter the dataset & save it to disk maybe?
    groub by `each parameter` and apply function
    save the results for each parameter - function combination list
    
def save_otpimal_stuff()
    open the json
    save under date range key
    
results_json = {
    'date currently at': {
        '1-month': {
            'mean': {
                'sma': ..,
                'rsi close': ..
                ...
            },
            'order': ..
            'linear': ..
        }
        '3-month': ..
        '6-month': ..
        '12-month': ..
        'max': ..
    }
}

If a date range is empty, the next one in line will be selected. If all are empty, it will use the default params.
It does NOT account for how many trades there are out there?


### Imports and variables

In [7]:
import datetime 
import json
from datetime import timedelta
from dateutil.relativedelta import relativedelta
from dateutil import rrule

import pandas as pd
import tqdm
from matplotlib import pyplot
import seaborn as sns
import dask.dataframe as dd

# TIMEFRAME = 'daily'
TIMEFRAME = 'weekly'

# FRAMEWORK = 'dask'
FRAMEWORK = 'pandas'

# PERC_CHANGE_METHOD = 'mean'
PERC_CHANGE_METHOD = 'order aware'

FIRST_DATE = datetime.datetime(2000, 1, 1)
LAST_DATE = datetime.datetime(2020, 12, 31)

FORWARD_OPT_RESULTS_JSON = f'results/{TIMEFRAME}/forward_opt_results.json'

### Utils

In [8]:
def get_sample(the_df):
    print(f'There are {len(the_df)} rows in the dataframe/series')
    try:
        res = the_df.sample(frac=5)
    except ValueError:
        res = the_df.sample(3)
    
    return res

In [9]:
def save_results(key, results_for_key):
    with open(FORWARD_OPT_RESULTS_JSON, 'r') as f:
        results_dict = json.loads(f.read())
        
    results_dict[key] = results_for_key
    
    with open(FORWARD_OPT_RESULTS_JSON, 'w') as f:
        f.write(json.dumps(results_dict))

# save_results('a', {1:2,3:4})

In [20]:
def get_pair_entry(start_date, end_date):
    return (
        f"{start_date.strftime('%y-%m')}-to-{end_date.strftime('%y-%m')}",
        start_date,
        end_date
    )

def generate_date_ranges():
    resulting_dict = {}
    for end_date in rrule.rrule(rrule.MONTHLY, dtstart=FIRST_DATE, until=LAST_DATE):
        resulting_dict[end_date.strftime('%y-%m')] = {
#             '1-month': get_pair_entry(end_date + relativedelta(months=-1), end_date),
#             '3-month': get_pair_entry(end_date + relativedelta(months=-3), end_date),
            '6-month': get_pair_entry(end_date + relativedelta(months=-6), end_date),
            '12-month': get_pair_entry(end_date + relativedelta(months=-12), end_date),
            'max': get_pair_entry(FIRST_DATE, end_date)
        }

    return resulting_dict


def get_parsed_date_ranges():
    with open(FORWARD_OPT_RESULTS_JSON, 'r') as f:
        results_dict = json.loads(f.read())
    
    return list(results_dict.keys())

# for k,v in generate_date_ranges().items():
#     print(k)
#     print(v)
# #     range_name, key, start_date, end_date = v
# #     print(range_name, key, start_date, end_date)
#     raise Exception

### Load the dataframe

In [21]:
if TIMEFRAME == 'daily':
    path = 'dataset/daily/'
    column_names = [
        "uid",
        "ticker",
        "date",
        "price_open",
        "maperiod",
        "rsi_open_period",
        "adx8",
        "adx16",
        "adx32",
        "ppo8",
        "ppo16",
        "ppo32",
        "stochastic8",
        "stochastic16",
        "stochastic32",
        "price_sell",
        "days_ago_close_period",
        "rsi_close_period",
    ]
elif TIMEFRAME == 'weekly':
    path = 'dataset/weekly/'    
    column_names = [
        "uid",
        "ticker",
        "date",
        "price_open",
        "maperiod",
        "rsi_open_period",
        "adx3",
        "adx6",
        "adx9",
        "ppo3",
        "ppo6",
        "ppo9",
        "stochastic3",
        "stochastic6",
        "stochastic9",
        "price_sell",
        "days_ago_close_period",
        "rsi_close_period",
    ]

if FRAMEWORK == 'pandas':
    d = pd
    f = f"{path}/all_results.csv"
elif FRAMEWORK == 'dask':
    d = dd
    f = f"{path}/*.csv"

# Read CSV file and assign the right column names
# Comment the first two rows if you don't want to use the entire dataset, and instead just work on a sample
df = d.read_csv(
    f,
    names=column_names,
)

In [22]:
get_sample(df)

There are 42596 rows in the dataframe/series


,uid,ticker,date,price_open,maperiod,rsi_open_period,adx3,adx6,adx9,ppo3,ppo6,ppo9,stochastic3,stochastic6,stochastic9,price_sell,days_ago_close_period,rsi_close_period
35421,ccc8ada9-c05e-11eb-854a-95fa4750c8c5,AXP,2010-04-29,37.68,45,2,71.691926,50.991281,39.927642,20.230539,18.314953,16.178059,50.973455,67.434881,74.998105,34.75,3,57.141888
38515,12fb551a-c05c-11eb-8546-95fa4750c8c5,MMM,2010-10-28,63.72,45,2,67.537864,33.040950,21.443295,7.075165,7.220042,6.567182,62.700871,67.721525,76.399782,64.43,0,34.142331
40196,f53d0808-c05d-11eb-8548-95fa4750c8c5,MO,2007-12-27,10.96,35,2,40.209478,39.280942,30.080929,8.106431,7.539823,6.494510,46.080369,73.333333,74.873512,11.40,1,56.027134


### Compute the change percentage
And drop the price_sell and price_open columns because they are no longer used

In [23]:
# Add the percentage change for the price
df["perc_change"] = (df["price_sell"] - df["price_open"]) / df["price_open"] * 100

In [24]:
# The prices can now be dropped since I won't need them anymore
df = df.drop(["price_sell", "price_open"], axis=1)

In [25]:
get_sample(df)

There are 42596 rows in the dataframe/series


,uid,ticker,date,maperiod,rsi_open_period,adx3,adx6,adx9,ppo3,ppo6,ppo9,stochastic3,stochastic6,stochastic9,days_ago_close_period,rsi_close_period,perc_change
4284,905c763a-c05c-11eb-8547-95fa4750c8c5,ADBE,2016-10-27,35,2,31.639358,28.765381,22.444453,7.598029,7.476459,6.679736,29.680905,55.724114,74.290852,1,19.380850,-1.709321
19121,adea7624-c05d-11eb-8548-95fa4750c8c5,ALL,2017-04-06,40,2,43.634043,54.493731,50.981378,9.803131,9.560990,8.716995,43.771324,45.073410,73.227699,5,90.629604,5.983135
21407,d7726b53-c05c-11eb-8547-95fa4750c8c5,AMD,2017-04-27,40,4,49.203109,29.563318,32.337276,14.433520,21.013207,22.954396,39.941173,27.520637,21.769234,2,32.299515,4.812320


# Strategy explanation
## Initial strategy parameters
### SMA
The simple moving average of close prices for the close price of the previous day to be above so that we have the first buy signal
### RSI open
The relative strength index lower barrier to break for a buy signal
### RSI close
The relative strength index barrier that, if it breaks above, indicates a sell signal
### Days to wait before closing a trade
If the RSI close barrier is not reached, how long to hold the trade before selling.

## Forward optimizations goals 
Find the best parameter combination for this strategy by exploring relatively close range of values for each of the previously mentioned parameters.
It's called 'forward' because we already have a strategy defined but we just tweak it's parameters in the hopes of improving results.
This will be explored next, and the focus on 'backward' optimization will fall into a later part of the notebook.


##### It is important to note how the dataset was generated, only the RSI open and SMA indicators were considered and 20 days were recorded.

### Optimization utilities
The idea is simple:

    1. Groub py a field
    2. Find the best value for 'perc_change'
    3. Return the best value for the field that was grouped by
    
The problem is step 2. How do you compute the best perc_change value?
Ideas:

    * The simplest approach is to use the mean. It's easy to compute but doesn't show us the real picture because it doesn't account for the order of trades                     

    * Another option may be to apply these trades in order to one unit. This doesn't account for random starting times.                                  
    
    * Another option may be to apply these trader in order to one unit, multiple times, each time dropping one entry from beggining to end. Finally, find the mean of these resulting values. The advantage is that it accounts for order but also for random starting points.                       

In [26]:
def mean_optimal_func(tmp_df_groupby):
    tmp_df = tmp_df_groupby["perc_change"].mean()
    max_value = tmp_df.idxmax()

    if FRAMEWORK == 'dask':
        max_value = max_value.compute()

    return max_value


def order_optimal_func(tmp_df_groupby):
    def outcome_computation(x):
        l = []
        for _, perc_change in x.sort_values(['date'])['perc_change'].iteritems():
            l.append(perc_change)

        results = []
        for i in range(len(l)):
            res = 1
            for j in range(i, len(l)):
                res += (res * l[j]) / 100
            results.append(res)

        return sum(results) / len(results)

    if FRAMEWORK == 'pandas':
        outcome_series = tmp_df_groupby.apply(outcome_computation)
    else:
        outcome_series = tmp_df_groupby.apply(outcome_computation, meta=('float'))

    res = outcome_series.idxmax()
    return res if FRAMEWORK == 'pandas' else res.compute()


def linear_optimal_func(df):
    pass


def compute_optimal_params_using_func(df, func):
    optimal_days_ago = int(func(df.groupby(['days_ago_close_period'])))
    df = df[df['days_ago_close_period'] == optimal_days_ago]
    
    optimal_rsi_open = int(func(df.groupby(['rsi_open_period'])))
    df = df[df['rsi_open_period'] == optimal_rsi_open]
    
    optimal_maperiod = int(func(df.groupby(['maperiod'])))
    df = df[df['maperiod'] == optimal_maperiod]
    
    result = {
        'days_ago_close_period': optimal_days_ago,
        'rsi_open_period': optimal_rsi_open,
        'maperiod': optimal_maperiod
    }
    
    return result 


def compute_optimal_results(df):
    return {
        'mean': compute_optimal_params_using_func(df, mean_optimal_func),
        'order': compute_optimal_params_using_func(df, order_optimal_func),
#         'linear': compute_optimal_params_using_func(filtered_df, linear_optimal_func),
    }


In [27]:
def filter_df_by_dates(df, start_date, end_date):
    return df[(df['date'] >= start_date.strftime('%Y-%m-%d')) & (df['date'] <= end_date.strftime('%Y-%m-%d'))]

In [28]:
STRATEGY_OUT_OF_THE_BOX_KWARGS = {
    'maperiod': 200,
    'rsi_open_period': 10,
    'days_ago_close_period': 10
} if TIMEFRAME == 'daily' else {
    'maperiod': 40,
    'rsi_open_period': 2,
    'days_ago_close_period': 2
}
DEFAULT_STRATEGY_KWARGS = {
    'mean': STRATEGY_OUT_OF_THE_BOX_KWARGS,
    'order': STRATEGY_OUT_OF_THE_BOX_KWARGS,
#     'linear': STRATEGY_OUT_OF_THE_BOX_KWARGS
}

In [41]:
# all_date_ranges = generate_date_ranges()
# parsed_date_ranges = get_parsed_date_ranges()
# unparsed_date_ranges = {k:v for k, v in all_date_ranges.items() if k not in parsed_date_ranges}
unparsed_date_ranges

{'00-01': {'6-month': ('99-07-to-00-01',
   datetime.datetime(1999, 7, 1, 0, 0),
   datetime.datetime(2000, 1, 1, 0, 0)),
  '12-month': ('99-01-to-00-01',
   datetime.datetime(1999, 1, 1, 0, 0),
   datetime.datetime(2000, 1, 1, 0, 0)),
  'max': ('00-01-to-00-01',
   datetime.datetime(2000, 1, 1, 0, 0),
   datetime.datetime(2000, 1, 1, 0, 0))},
 '00-02': {'6-month': ('99-08-to-00-02',
   datetime.datetime(1999, 8, 1, 0, 0),
   datetime.datetime(2000, 2, 1, 0, 0)),
  '12-month': ('99-02-to-00-02',
   datetime.datetime(1999, 2, 1, 0, 0),
   datetime.datetime(2000, 2, 1, 0, 0)),
  'max': ('00-01-to-00-02',
   datetime.datetime(2000, 1, 1, 0, 0),
   datetime.datetime(2000, 2, 1, 0, 0))},
 '00-03': {'6-month': ('99-09-to-00-03',
   datetime.datetime(1999, 9, 1, 0, 0),
   datetime.datetime(2000, 3, 1, 0, 0)),
  '12-month': ('99-03-to-00-03',
   datetime.datetime(1999, 3, 1, 0, 0),
   datetime.datetime(2000, 3, 1, 0, 0)),
  'max': ('00-01-to-00-03',
   datetime.datetime(2000, 1, 1, 0, 0),
   d

In [44]:
with tqdm.tqdm(total=len(unparsed_date_ranges)) as pbar:

    # Simple caching mechanism to speed things up for a lot of computations
    PREVIOUS_SIZE = None
    PREVIOUS_RES = None
    
    for date_value, date_ranges in unparsed_date_ranges.items():
        optimal_results = {}

        def optimize_range(df, range_name, default_if_empty):
            range_values = date_ranges[range_name]
            _, start_date, end_date = range_values
            
            filtered_df = filter_df_by_dates(df, start_date, end_date)
            size = filtered_df.shape[0] if FRAMEWORK == 'pandas' else filtered_df.shape[0].compute()

            if size == globals()['PREVIOUS_SIZE']:
                print(f'Using chache for {range_name} - {start_date} to {end_date}')
                return globals()['PREVIOUS_RES']
            
            # TODO: Does this account for statistical significant results?
            if size < 100:
                print(f'Empty or too small df from (size: {size}) {start_date} to {end_date}')
                res = default_if_empty
            else:
                print(f'Parsing {size} lines from {start_date} to {end_date}')
                res = compute_optimal_results(filtered_df)

            globals()['PREVIOUS_SIZE'] = size
            globals()['PREVIOUS_RES'] = res
            
            return res
                
        optimal_results['max'] = optimize_range(df, 'max', DEFAULT_STRATEGY_KWARGS)
        optimal_results['12-month'] = optimize_range(df, '12-month', optimal_results['max'])
        optimal_results['6-month'] = optimize_range(df, '6-month', optimal_results['12-month'])
#         optimal_results['3-month'] = optimize_range(df, '3-month', optimal_results['6-month'])
#         optimal_results['1-month'] = optimize_range(df, '1-month', optimal_results['3-month'])

        print(date_value)
        print(optimal_results)
        save_results(date_value, optimal_results)
        pbar.update()
    

  1%|          | 2/252 [00:00<00:13, 18.07it/s]

Empty or too small df from (size: 0) 2000-01-01 00:00:00 to 2000-01-01 00:00:00
Parsing 1482 lines from 1999-01-01 00:00:00 to 2000-01-01 00:00:00
Parsing 702 lines from 1999-07-01 00:00:00 to 2000-01-01 00:00:00
00-01
{'max': {'mean': {'maperiod': 40, 'rsi_open_period': 2, 'days_ago_close_period': 2}, 'order': {'maperiod': 40, 'rsi_open_period': 2, 'days_ago_close_period': 2}}, '12-month': {'mean': {'days_ago_close_period': 4, 'rsi_open_period': 6, 'maperiod': 50}, 'order': {'days_ago_close_period': 4, 'rsi_open_period': 2, 'maperiod': 35}}, '6-month': {'mean': {'days_ago_close_period': 3, 'rsi_open_period': 4, 'maperiod': 30}, 'order': {'days_ago_close_period': 3, 'rsi_open_period': 2, 'maperiod': 35}}}
Empty or too small df from (size: 54) 2000-01-01 00:00:00 to 2000-02-01 00:00:00
Parsing 1320 lines from 1999-02-01 00:00:00 to 2000-02-01 00:00:00
Parsing 588 lines from 1999-08-01 00:00:00 to 2000-02-01 00:00:00
00-02
{'max': {'mean': {'maperiod': 40, 'rsi_open_period': 2, 'days_ago

  2%|▏         | 6/252 [00:00<00:15, 15.87it/s]

Parsing 558 lines from 1999-10-01 00:00:00 to 2000-04-01 00:00:00
00-04
{'max': {'mean': {'days_ago_close_period': 4, 'rsi_open_period': 2, 'maperiod': 30}, 'order': {'days_ago_close_period': 4, 'rsi_open_period': 4, 'maperiod': 55}}, '12-month': {'mean': {'days_ago_close_period': 4, 'rsi_open_period': 6, 'maperiod': 45}, 'order': {'days_ago_close_period': 4, 'rsi_open_period': 2, 'maperiod': 55}}, '6-month': {'mean': {'days_ago_close_period': 3, 'rsi_open_period': 2, 'maperiod': 30}, 'order': {'days_ago_close_period': 3, 'rsi_open_period': 2, 'maperiod': 55}}}
Parsing 276 lines from 2000-01-01 00:00:00 to 2000-05-01 00:00:00
Parsing 1344 lines from 1999-05-01 00:00:00 to 2000-05-01 00:00:00
Parsing 444 lines from 1999-11-01 00:00:00 to 2000-05-01 00:00:00
00-05
{'max': {'mean': {'days_ago_close_period': 4, 'rsi_open_period': 2, 'maperiod': 30}, 'order': {'days_ago_close_period': 4, 'rsi_open_period': 4, 'maperiod': 55}}, '12-month': {'mean': {'days_ago_close_period': 4, 'rsi_open_peri

  3%|▎         | 8/252 [00:00<00:16, 15.05it/s]

...

Parsing 1146 lines from 2019-10-01 00:00:00 to 2020-10-01 00:00:00
Parsing 384 lines from 2020-04-01 00:00:00 to 2020-10-01 00:00:00
20-10
{'max': {'mean': {'days_ago_close_period': 5, 'rsi_open_period': 4, 'maperiod': 45}, 'order': {'days_ago_close_period': 5, 'rsi_open_period': 2, 'maperiod': 40}}, '12-month': {'mean': {'days_ago_close_period': 0, 'rsi_open_period': 4, 'maperiod': 40}, 'order': {'days_ago_close_period': 5, 'rsi_open_period': 2, 'maperiod': 55}}, '6-month': {'mean': {'days_ago_close_period': 5, 'rsi_open_period': 4, 'maperiod': 35}, 'order': {'days_ago_close_period': 5, 'rsi_open_period': 2, 'maperiod': 45}}}
Parsing 40836 lines from 2000-01-01 00:00:00 to 2020-11-01 00:00:00


100%|█████████▉| 251/252 [15:58<00:10, 10.33s/it]

Parsing 1068 lines from 2019-11-01 00:00:00 to 2020-11-01 00:00:00
Parsing 384 lines from 2020-05-01 00:00:00 to 2020-11-01 00:00:00
20-11
{'max': {'mean': {'days_ago_close_period': 5, 'rsi_open_period': 4, 'maperiod': 45}, 'order': {'days_ago_close_period': 5, 'rsi_open_period': 2, 'maperiod': 40}}, '12-month': {'mean': {'days_ago_close_period': 0, 'rsi_open_period': 4, 'maperiod': 40}, 'order': {'days_ago_close_period': 5, 'rsi_open_period': 2, 'maperiod': 30}}, '6-month': {'mean': {'days_ago_close_period': 5, 'rsi_open_period': 4, 'maperiod': 35}, 'order': {'days_ago_close_period': 5, 'rsi_open_period': 2, 'maperiod': 45}}}
Parsing 40950 lines from 2000-01-01 00:00:00 to 2020-12-01 00:00:00


100%|██████████| 252/252 [16:08<00:00,  3.84s/it]

Parsing 1002 lines from 2019-12-01 00:00:00 to 2020-12-01 00:00:00
Parsing 498 lines from 2020-06-01 00:00:00 to 2020-12-01 00:00:00
20-12
{'max': {'mean': {'days_ago_close_period': 5, 'rsi_open_period': 4, 'maperiod': 45}, 'order': {'days_ago_close_period': 5, 'rsi_open_period': 2, 'maperiod': 40}}, '12-month': {'mean': {'days_ago_close_period': 0, 'rsi_open_period': 4, 'maperiod': 40}, 'order': {'days_ago_close_period': 5, 'rsi_open_period': 2, 'maperiod': 45}}, '6-month': {'mean': {'days_ago_close_period': 5, 'rsi_open_period': 4, 'maperiod': 35}, 'order': {'days_ago_close_period': 5, 'rsi_open_period': 2, 'maperiod': 45}}}
